In [1]:
import mlflow
from mlflow.tracking import MlflowClient
from datetime import datetime
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import joblib
import os
from sklearn.model_selection import GridSearchCV

# Funciones para enviar data a base de datos

In [2]:
# ------------------------------------ Load raw data --------------------------------------
def load_csv_to_mysql():
    try:
        csv_path = 'penguins_lter.csv'
        df = pd.read_csv(csv_path)

        engine = create_engine('mysql+pymysql://user:password@db:3306/db_jupyter')

        # Guardar en MySQL
        df.to_sql("penguins_original", con=engine, if_exists='replace', index=False)
        print("✅ Datos originales cargados en MySQL.")
    
    except Exception as e:
        print(f"❌ Error en load_csv_to_mysql: {e}")

# ------------------------------------ Load transformed data --------------------------------------
def preprocesamiento():
    try:
        engine = create_engine("mysql+pymysql://user:password@db:3306/db_jupyter")

        # Leer los datos de MySQL con pd.read_sql en lugar de pd.read_sql_table
        df = pd.read_sql("SELECT * FROM penguins_original", con=engine)

        # Transformación de "Date Egg" a día del año (manejo de NaN)
        if "Date Egg" in df.columns:
            df["Date Egg"] = pd.to_datetime(df["Date Egg"], errors="coerce").dt.dayofyear
            df["Date Egg"].fillna(df["Date Egg"].median(), inplace=True)

        # Eliminar columnas irrelevantes si existen
        cols_to_drop = ["studyName", "Sample Number", "Individual ID", "Comments"]
        df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

        # Manejo de valores nulos en columnas numéricas
        num_cols = ["Culmen Length (mm)", "Culmen Depth (mm)", "Flipper Length (mm)", "Body Mass (g)", "Delta 15 N (o/oo)", "Delta 13 C (o/oo)"]
        for col in num_cols:
            if col in df.columns:
                df[col].fillna(df[col].mean(), inplace=True)

        # Manejo de valores nulos en columnas categóricas
        cat_cols = ["Species", "Region", "Island", "Stage", "Clutch Completion", "Sex"]
        for col in cat_cols:
            if col in df.columns:
                df[col].fillna("Desconocido", inplace=True)

        # One-Hot Encoding para variables categóricas si existen
        df = pd.get_dummies(df, columns=[col for col in cat_cols if col in df.columns], drop_first=True)

        # Selección de variables predictoras y objetivo
        feature_cols = ["Culmen Length (mm)", "Culmen Depth (mm)", "Flipper Length (mm)"]
        target_col = "Body Mass (g)"

        if not all(col in df.columns for col in feature_cols + [target_col]):
            raise ValueError("⚠️ Algunas columnas clave no están en los datos procesados.")

        X = df[feature_cols]
        y = df[target_col]

        # Dividir en entrenamiento y prueba (80%-20%)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Guardar los conjuntos en MySQL
        X_train.to_sql("train_data_X", con=engine, if_exists="replace", index=False)
        X_test.to_sql("test_data_X", con=engine, if_exists="replace", index=False)
        y_train.to_sql("train_data_y", con=engine, if_exists="replace", index=False)
        y_test.to_sql("test_data_y", con=engine, if_exists="replace", index=False)

        print("✅ Preprocesamiento completo. Tablas 'train_data_X', 'train_data_y', 'test_data_X' y 'test_data_y' creadas en MySQL.")

    except Exception as e:
        print(f"❌ Error en preprocesamiento: {e}")

In [3]:
#truncate_all_tables()
load_csv_to_mysql()
preprocesamiento()

✅ Datos originales cargados en MySQL.


/tmp/ipykernel_22/328991044.py:26: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date Egg"] = pd.to_datetime(df["Date Egg"], errors="coerce").dt.dayofyear
/tmp/ipykernel_22/328991044.py:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Date Egg"].fillna(df["Date Egg"].median(), inplace=True)
/tmp/ipykernel_22/328991044.py:37: FutureWarning: A value is trying to be set on a copy of 

✅ Preprocesamiento completo. Tablas 'train_data_X', 'train_data_y', 'test_data_X' y 'test_data_y' creadas en MySQL.


# Registrar modelos en MLFlow

In [5]:
engine = create_engine("mysql+pymysql://user:password@db:3306/db_jupyter")

    
# Separar variables predictoras y objetivo
X_train = pd.read_sql_table("train_data_X", engine)
y_train = pd.read_sql_table("train_data_y", engine).values.ravel()
X_test = pd.read_sql_table("test_data_X", engine)
y_test = pd.read_sql_table("test_data_y", engine).values.ravel()

In [6]:
client = MlflowClient()
for exp in client.search_experiments(view_type="ALL"):
    print(exp.experiment_id, exp.name, exp.lifecycle_stage)

3 Gradient_Boosting_looking_for_the_best_model active
2 Random_forest_looking_for_the_best_model active
1 mlflow_tracking_pinguins_proofs active
0 Default active


In [10]:
#client.restore_experiment(experiment_id="1")  # reemplaza con el ID real

In [11]:
# connects to the Mlflow tracking server that you started above
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://192.168.1.10:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://192.168.1.10:5000")
mlflow.set_experiment("mlflow_tracking_pinguins_proofs")

mlflow.autolog(log_model_signatures=True, log_input_examples=True)

# run description (just metadata)
desc = "the simplest possible example"

# executes the run
with mlflow.start_run(run_name="Random_forest_no_params", description=desc) as run:
    # Entrenar RandomForest
    rf_model = RandomForestRegressor()
    rf_model.fit(X_train, y_train)
    
#mlflow.end_run()

2025/03/14 15:00:34 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [12]:
print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

tracking uri: http://192.168.1.10:5000
artifact uri: s3://mlflows3/artifacts/1/fab3ef7f93584a2b9c1bc633b506b72a/artifacts


In [16]:
# connects to the Mlflow tracking server that you started above
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://192.168.1.10:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://192.168.1.10:5000")
mlflow.set_experiment("mlflow_tracking_pinguins_proofs")

mlflow.autolog(log_model_signatures=True, log_input_examples=True)

# run description (just metadata)
desc = "the simplest possible example"

# executes the run
with mlflow.start_run(run_name="Gradient_boost_no_params", description=desc) as run:
    # Entrenar GradietBoost
    gb_model = GradientBoostingRegressor()
    gb_model.fit(X_train, y_train)
    
#mlflow.end_run()  

2025/03/14 15:23:02 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


In [20]:
# connects to the Mlflow tracking server that you started above
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://192.168.1.10:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://192.168.1.10:5000")
mlflow.set_experiment("Random_forest_looking_for_the_best_model")

mlflow.autolog(log_model_signatures=True, log_input_examples=True)

# run description (just metadata)
desc = "Random_forest_grid_search"

params = {
  "n_estimators": [33, 66, 200],
  "max_depth": [2, 4, 6],
  "max_features": [3, 4, 5]
}

rf_model = RandomForestRegressor()
searcher = GridSearchCV(estimator=rf_model, param_grid=params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

with mlflow.start_run(run_name="autolog_with_grid_search") as run:
    searcher.fit(X_train, y_train)
    
#mlflow.end_run()  

2025/03/14 15:40:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/03/14 15:40:52 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/03/14 15:41:12 INFO mlflow.sklearn.utils: Logging the 5 best runs, 22 runs will be omitted.


In [21]:
# connects to the Mlflow tracking server that you started above
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://192.168.1.10:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

mlflow.set_tracking_uri("http://192.168.1.10:5000")
mlflow.set_experiment("Gradient_Boosting_looking_for_the_best_model")

mlflow.autolog(log_model_signatures=True, log_input_examples=True)

# run description (just metadata)
desc = "Gradient_Boosting_grid_search"

param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 1.0]
}

gb_model = GradientBoostingRegressor()
searcher = GridSearchCV(estimator=gb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

with mlflow.start_run(run_name="autolog_with_grid_search") as run:
    searcher.fit(X_train, y_train)
    
#mlflow.end_run()  

2025/03/14 16:00:58 INFO mlflow.tracking.fluent: Experiment with name 'Gradient_Boosting_looking_for_the_best_model' does not exist. Creating a new experiment.
2025/03/14 16:00:58 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/03/14 16:01:07 INFO mlflow.sklearn.utils: Logging the 5 best runs, 31 runs will be omitted.


# Probar inferencia

**Probar inferencia mejor modelo de random forest**

In [31]:
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://192.168.1.10:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

# connect to mlflow
mlflow.set_tracking_uri("http://192.168.1.10:5000")

model_name = "Best_random_forest"

# logged_model = 'runs:/71428bebed2b4feb9635714ea3cdb562/model'
model_production_uri = "models:/{model_name}/production".format(model_name=model_name)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri=model_production_uri)
loaded_model
example_test = X_test.iloc[0].to_frame().T
#print(example_test)
print('real: ', y_test[0])
print('prediction: ', loaded_model.predict(example_test)[0])

real:  3550.0
prediction:  3932.1369682485915


/usr/local/lib/python3.9/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


**Probar inferencia mejor modelo de gradient boosting**


In [10]:
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://192.168.1.10:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

# connect to mlflow
mlflow.set_tracking_uri("http://192.168.1.10:5000")

model_name = "Best_gradient_boosting"

# logged_model = 'runs:/71428bebed2b4feb9635714ea3cdb562/model'
model_production_uri = "models:/{model_name}/production".format(model_name=model_name)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(model_uri=model_production_uri)
loaded_model
example_test = X_test.iloc[0].to_frame().T
#print(example_test)
print('real: ', y_test[0])
print('prediction: ', loaded_model.predict(example_test)[0])

real:  3550.0
prediction:  3772.698561875597


/usr/local/lib/python3.9/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [12]:
X_test.iloc[0].to_frame()

,0
Culmen Length (mm),50.9
Culmen Depth (mm),19.1
Flipper Length (mm),196.0


In [7]:
client = MlflowClient()

# List all registered models using search_registered_models
registered_models = client.search_registered_models()

# Print the names of the models
for model in registered_models:
    print(model.name)

Best_gradient_boosting
Best_random_forest


In [8]:
print(registered_models)

[<RegisteredModel: aliases={}, creation_timestamp=1741986372411, description='', last_updated_timestamp=1741986387485, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1741986372476, current_stage='Production', description='', last_updated_timestamp=1741986387485, name='Best_gradient_boosting', run_id='3d57fbf14a0c44a9b259d4283e1f0867', run_link='', source='s3://mlflows3/artifacts/3/3d57fbf14a0c44a9b259d4283e1f0867/artifacts/best_estimator', status='READY', status_message='', tags={}, user_id='', version='1'>], name='Best_gradient_boosting', tags={}>, <RegisteredModel: aliases={}, creation_timestamp=1741985542441, description='', last_updated_timestamp=1741986339075, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1741985542518, current_stage='Production', description='', last_updated_timestamp=1741985827221, name='Best_random_forest', run_id='0160c204d40248078897b9fac5b33c68', run_link='', source='s3://mlflows3/artifacts/2/0160c204d40248078897b9fac5b33c68/